In [9]:
import requests
import abc

In [11]:
class SpaceTrack(abc.ABC):
    """ Provides an API for making POST requests to space-track.org """
    def __init__(self, username: str, password: str, fmt: str=None):
        """ Initializes the API.
        
        This API class can be used to build and submit requests to space-track.org.
        
        Args:
            username: your space-track.org username.
            password: the associated password.
        
        Kwargs:
            fmt: string specifying format for returned message. Can be one of
            'xml', 'json', 'html', 'csv', 'tle', '3le', 'kvn', or None. None
            is the same as 'json'. Default is None.
        
        Raises:
            ValueError: if provided fmt is not one of the specified options.
        """
        valid_fmt = ['xml', 'json', 'html', 'csv', 'tle', '3le', 'kvn']
        if fmt is not None:
            if fmt.lower() not in valid_fmt:
                raise ValueError("fmt must be one of 'xml', 'json', 'html', \
                                 'csv', 'tle', '3le', 'kvn', or None.")
        else:
            fmt = 'json'
        self.fmt = fmt.lower()
        self.username = username
        self.password = password
        self.base = 'https://space-track.org/'  # base URL for requests
        self.login_url = 'https://www.space-track.org/ajaxauth/login'  # login URL
        self.logout_url = 'https://www.space-track.org/ajaxauth/logout'
        self.query = []  # placeholder for our query string
    
    def _logout(self) -> requests.models.Response:
        """ Logs out of the space-track.org session.
        
        Returns:
            Response from space-track.org
        """
        res = requests.post(self.logout_url)
        if not res.ok:
            print('Error logging out! Status code {}'.format(res.status_code))
        return res
    
    def _basic(self):
        """ Adds 'basicspacedata' to the query. """
        self.query.append('basicspacedata')
    
    def _query(self):
        """ Builds the start of a query. """
        self._basic()
        self.query.append('query')
        
    def _compile_query(self) -> str:
        """ Compiles the query (list of strings) into a '/'-separated string.
        
        Returns:
            The query string with each element separated by a forward-slash.
        """
        if self.fmt not in self.query:
            self.query.extend(['format', self.fmt])
        return '/'.join(self.query)
    
    def print_query(self):
        """ Prints the query to the screen.
        
        Note that at this stage the format string has not been applied to
        the query. This string gets applied at the very end prior to
        submitting the POST request.
        """
        print('/'.join(self.query))
    
    def submit(self) -> requests.models.Response:
        """ Submits the generated query to space-track.org and returns the response.
        
        Returns:
            Response from space-track.org
        """
        payload = {'identity': self.username,
                   'password': self.password,
                   'query': self._compile_query()}
        res = requests.post(self.login_url, data=payload)
        if not res.ok:
            print('Error posting request! Status code {}'.format(res.status_code))
        self._logout()
        return res

In [ ]:
class TLE(SpaceTrack):
    """ TLE request from space-track.org. """
    def __init__(self, *args, **kwargs):
        """ Initiates a tle request. """
        self.keys = ['COMMENT', 'ORIGINATOR', 'NORAD_CAT_ID', 'OBJECT_NAME',
                     'OBJECT_TYPE', 'CLASSIFICATION_TYPE', 'INTLDES', 'EPOCH',
                     'MEAN_MOTION', 'ECCENTRICITY',
                     'INCLINATION', 'MEAN_ANOMALY', 'PERIOD']
        self._query()
        self.query.extend('class', 'tle')
        super().__init__(*args, **kwargs)
        
    def norad_cat_id(self, id_number: int):
        """ Specifies the NORAD_CAT_ID field in the query.
        
        Use this to find a specific satellite in the catalog.
        
        Args:
            id_number: the catalog ID number of the satellite.
        """
        self.query.extend('NORAD_CAT_ID', str(id_number))
    
    def epoch(self, start: str=None, end: str=None, equal: bool=False):
        """ Specifies a date range for the EPOCH field in the query.
        
        If only a start date is specified and 'equal' is set to 'False',
        the query will look for epochs on or after the start date. If
        both a start and end date are specified, the query will look
        for the range between those dates. If only an end date is
        specified and 'equal' is set to 'False', the query will look for
        epochs on or before the end date.
        
        'equal' can be set to 'True', in which case it will look for
        epochs exactly on the start date (if specified) or end date
        if no start date is specified.
        
        All dates are datetime-formatted strings of the form:
        
            YYYY-MM-DD HH:mm:ss
            
        They can include just the year; or year + month; or year, month,
        and day as well. Other combinations may be possible; further
        testing is required.
        
        Kwargs:
            start: start date as a datetime-formatted string.
            end: end date as a datetime-formatted string.
            equal: if False, looks for dates greater than the start date
                if only a start date is specified, less than the end date
                if only an end date is specified, and between the start
                and end dates if both are specified.
        """
        self.query.append('EPOCH')
        if equal:
            if start is not None:
                self.query.append(start)
            elif end is not None:
                self.query.append(end)
            else:
                raise ValueError("Either 'start' or 'end' must be specified \
                                  if 'equal' is True.")
        else:
            if start is not None and end is not None:
                date_string = start + '--' + end
                self.query.append(date_string)
            elif start is not None and end is None:
                date_string = '>' + start
                self.query.append(date_string)
            elif start is None and end is not None:
                date_string = '<' + end
                self.query.append(date_string)
            else:
                raise ValueError("Either 'start' or 'end' or both must be \
                                  specified if 'equal' is False.")
   

In [ ]:
 
class TleLatest(SpaceTrack):
    """ TLE Latest request from space-track.org. """
    def __init__(self, *args, **kwargs):
        """ Initiates a tle_latest request. """
        self._query()
        self.query.extend('class', 'tle_latest')
        super().__init__(*args, **kwargs)
    
class TlePublish(SpaceTrack):
    """ TLE Publish request from space-track.org. """
    def __init__(self, *args, **kwargs):
        """ Initiates a tle_publish request. """
        self._query()
        self.query.extend('class', 'tle_publish')
        super().__init__(*args, **kwargs)
    
class BoxScore(SpaceTrack):
    def __init__(self, *args, **kwargs):
        """ Initiates a boxscore request. """
        self._query()
        self.query.extend('class', 'boxscore')
        super().__init__(*args, **kwargs)
        
class SatCat(SpaceTrack):
    def __init__(self, *args, **kwargs):
        """ Initiates a satcat request. """
        self._query()
        self.query.extend('class', 'satcat')
        super().__init__(*args, **kwargs)
        
class LaunchSite(SpaceTrack):
    def __init__(self, *args, **kwargs):
        """ Initiates a launch_site request. """
        self._query()
        self.query.extend('class', 'launch_site')
        super().__init__(*args, **kwargs)
        
class SatCatChange(SpaceTrack):
    def __init__(self, *args, **kwargs):
        """ Initiates a satcat_change request. """
        self._query()
        self.query.extend('class', 'satcat_change')
        super().__init__(*args, **kwargs)
        
class SatCatDebut(SpaceTrack):
    def __init__(self, *args, **kwargs):
        """ Initiates a satcat_debut request. """
        self._query()
        self.query.extend('class', 'satcat_debut')
        super().__init__(*args, **kwargs)
        
class Decay(SpaceTrack):
    def __init__(self, *args, **kwargs):
        """ Initiates a decay request. """
        self._query()
        self.query.extend('class', 'decay')
        super().__init__(*args, **kwargs)
        
class Announcement(SpaceTrack):
    def __init__(self, *args, **kwargs):
        """ Initiates a announcement request. """
        self._query()
        self.query.extend('class', 'announcement')
        super().__init__(*args, **kwargs)
        
class CDM(SpaceTrack):
    def __init__(self, *args, **kwargs):
        """ Initiates a cdm request. """
        self._query()
        self.query.extend('class', 'cdm')
        super().__init__(*args, **kwargs)
        
class Organization(SpaceTrack):
    def __init__(self, *args, **kwargs):
        """ Initiates a organization request. """
        self._query()
        self.query.extend('class', 'organization')
        super().__init__(*args, **kwargs)